In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt
from pathlib import Path

In [2]:
# Tuning Values

HPARAMS_DEFAULTS = 0
HPARAMS_PAPER = 1
HPARAMS_CUSTOM = 2
training_type = HPARAMS_PAPER


OP_RMSPROP = 0
OP_SGD = 1
op_type = OP_RMSPROP

if (training_type == HPARAMS_DEFAULTS):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 32   # X
    dropout_rate = 0.4              # X
    batch_size = 32                 # X 
    optimizer_learning_rate = 0.01  # X  
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "relu"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30
    
elif (training_type == HPARAMS_PAPER):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 256  # X
    dropout_rate = 0.4              # X
    batch_size = 2275               # X 
    optimizer_learning_rate = 3e-4  # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.9
    learning_rate_decay = 0.2       # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 60
    
elif (training_type == HPARAMS_CUSTOM):
    num_hidden_layers = 1
    num_hidden_layer_neurons = 135     # X
    dropout_rate = 0.2596709650832801  # X
    batch_size = 2275                  # X 
    optimizer_learning_rate = 0.0029044390072926946   # X // 3e-4, or 3*(10^-4) = 0.0003
    gradient_decay_rho = 0.7437864402038941
    learning_rate_decay = 0.39953909944182836         # X
    hidden_layer_activation_function = "tanh"
    #-----
    op_type = OP_RMSPROP
    training_epochs = 30

'''
Paper Results: 
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7001 - loss: 0.5711 - val_accuracy: 0.9895 - val_loss: 0.1341

Custom Results:
Epoch 30/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8090 - loss: 0.4114 - val_accuracy: 0.9987 - val_loss: 0.0293


Trial 05 summary
Hyperparameters:
n_hidden: 1
n_neurons: 135
learning_rate: 0.0029044390072926946
dropout_rate: 0.2596709650832801
optimizer: rmsprop
rho: 0.7437864402038941
learning_rate_decay: 0.39953909944182836
decay_steps: 9028
Score: 1.0
'''

output_layer_activation_function = "sigmoid"
tf.keras.utils.set_random_seed(42)


use_gpu = True
if not use_gpu:
    import os
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
DATA_PREPROCESSING_SCOTTY = 0
DATA_PREPROCESSING_DAMITA = 1
DATA_PREPROCESSING_TYPE = DATA_PREPROCESSING_DAMITA

train_path = ""
test_path  = ""
names = []
#names = ["Match Result", "Blue Top Player", "Blue Jungle Player", "Blue Middle Player", "Blue ADC Player", "Blue Support Player", "Red Top Player", "Red Jungle Player", "Red Middle Player", "Red ADC Player", "Red Support Player", "Blue Top Champion", "Blue Jungle Champion", "Blue Middle Champion", "Blue ADC Champion", "Blue Support Champion", "Red Top Champion", "Red Jungle Champion", "Red Middle Champion", "Red ADC Champion", "Red Support Champion", "Blue Player Cooperation", "Red Player Cooperation", "Blue vs Red Player", "Blue Champion Cooperation", "Red Champion Cooperation", "Blue vs Red Champion", "Blue Team Win Rate When Blue", "Red Team Win Rate When Red"]

if (DATA_PREPROCESSING_TYPE == DATA_PREPROCESSING_SCOTTY):
    train_path = Path("../../feature_data/train/featureInput_noChampionRole_ZSCORED.csv")
    test_path  = Path("../../feature_data/test/featureInput_noChampionRole_ZSCORED.csv")
    names = ["bResult", "btPlayerRole", "bjPlayerRole", "bmPlayerRole", "baPlayerRole", "bsPlayerRole", "rtPlayerRole", "rjPlayerRole", "rmPlayerRole", "raPlayerRole", "rsPlayerRole", "btPlayerChampion", "bjPlayerChampion", "bmPlayerChampion", "baPlayerChampion", "bsPlayerChampion", "rtPlayerChampion", "rjPlayerChampion", "rmPlayerChampion", "raPlayerChampion", "rsPlayerChampion", "bCoopPlayer", "rCoopPlayer", "vsPlayer", "bCoopChampion", "rCoopChampion", "vsChampion", "bTeamColor", "rTeamColor"]
else:
    train_path = Path("../damita/trial_train_data.csv")
    test_path  = Path("../damita/trial_test_data.csv")
    names = ["btPlayerRole", "bjPlayerRole", "bmPlayerRole", "baPlayerRole", "bsPlayerRole", "rtPlayerRole", "rjPlayerRole", "rmPlayerRole", "raPlayerRole", "rsPlayerRole", "btPlayerChampion", "bjPlayerChampion", "bmPlayerChampion", "baPlayerChampion", "bsPlayerChampion", "rtPlayerChampion", "rjPlayerChampion", "rmPlayerChampion", "raPlayerChampion", "rsPlayerChampion", "bCoopPlayer", "rCoopPlayer", "bCoopChampion", "rCoopChampion", "vsPlayer", "vsChampion", "bTeamColor", "rTeamColor", "bResult"]


# Load our pre-processed training and test sets. 
# The skiprows value just removes the first row of the CSV which are labels instead of data.
train_data = (pd.read_csv(train_path, names=names, skiprows=1))
test_data  = (pd.read_csv( test_path, names=names, skiprows=1))

temp_data = (pd.read_csv(train_path, names=names, skiprows=1))


In [4]:
temp_data = temp_data.drop(['bResult'], axis=1)
temp_data.head()

,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,rsPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,bCoopChampion,rCoopChampion,vsPlayer,vsChampion,bTeamColor,rTeamColor
1,0.289738,-0.257036,1.060552,0.377305,0.278699,-0.133841,0.350029,-0.112189,0.713630,-0.216989,...,1.126587,0.690359,0.441921,0.096551,0.134805,1.112304,-0.175703,0.510275,0.121245,-0.151772
2,-0.263885,-0.766333,-0.260659,-0.853495,-0.950215,-0.332546,3.260819,-1.208941,-0.316147,-1.474715,...,-0.030390,0.346412,-0.698948,-0.656359,-1.708774,0.131480,-1.581668,0.359287,-0.173614,2.230129
3,-0.358951,0.020762,0.699159,-0.409335,0.500586,0.974318,0.453523,0.311440,0.535215,-1.244525,...,-0.030390,0.870778,-0.171815,-0.720948,0.224274,-0.419115,0.677840,0.532583,1.890399,-1.884064
4,-0.712056,0.159661,-0.022286,-0.039075,0.478613,-0.214329,0.932187,-0.196032,3.315169,0.161031,...,-1.157802,0.870778,0.250172,1.716844,0.312525,0.042050,-0.818070,0.405424,-2.237627,2.230129
5,2.873311,0.535202,0.182033,1.240093,0.421766,-0.473280,-0.463145,-0.570401,-0.762098,-0.460782,...,-0.030390,0.346412,0.469980,-0.334529,0.588069,-0.805653,1.530957,1.675086,1.890399,-0.512666


In [5]:
train_data.head()

,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,rsPlayerRole,...,rsPlayerChampion,bCoopPlayer,rCoopPlayer,bCoopChampion,rCoopChampion,vsPlayer,vsChampion,bTeamColor,rTeamColor,bResult
1,0.289738,-0.257036,1.060552,0.377305,0.278699,-0.133841,0.350029,-0.112189,0.713630,-0.216989,...,0.690359,0.441921,0.096551,0.134805,1.112304,-0.175703,0.510275,0.121245,-0.151772,1.0
2,-0.263885,-0.766333,-0.260659,-0.853495,-0.950215,-0.332546,3.260819,-1.208941,-0.316147,-1.474715,...,0.346412,-0.698948,-0.656359,-1.708774,0.131480,-1.581668,0.359287,-0.173614,2.230129,0.0
3,-0.358951,0.020762,0.699159,-0.409335,0.500586,0.974318,0.453523,0.311440,0.535215,-1.244525,...,0.870778,-0.171815,-0.720948,0.224274,-0.419115,0.677840,0.532583,1.890399,-1.884064,1.0
4,-0.712056,0.159661,-0.022286,-0.039075,0.478613,-0.214329,0.932187,-0.196032,3.315169,0.161031,...,0.870778,0.250172,1.716844,0.312525,0.042050,-0.818070,0.405424,-2.237627,2.230129,0.0
5,2.873311,0.535202,0.182033,1.240093,0.421766,-0.473280,-0.463145,-0.570401,-0.762098,-0.460782,...,0.346412,0.469980,-0.334529,0.588069,-0.805653,1.530957,1.675086,1.890399,-0.512666,1.0


In [6]:
len(train_data)

6824

In [7]:
len(test_data)

758

In [8]:
train_data.shape

(6824, 29)

In [9]:
test_data.shape

(758, 29)

In [10]:
train_data.dtypes

btPlayerRole        float64
bjPlayerRole        float64
bmPlayerRole        float64
baPlayerRole        float64
bsPlayerRole        float64
rtPlayerRole        float64
rjPlayerRole        float64
rmPlayerRole        float64
raPlayerRole        float64
rsPlayerRole        float64
btPlayerChampion    float64
bjPlayerChampion    float64
bmPlayerChampion    float64
baPlayerChampion    float64
bsPlayerChampion    float64
rtPlayerChampion    float64
rjPlayerChampion    float64
rmPlayerChampion    float64
raPlayerChampion    float64
rsPlayerChampion    float64
bCoopPlayer         float64
rCoopPlayer         float64
bCoopChampion       float64
rCoopChampion       float64
vsPlayer            float64
vsChampion          float64
bTeamColor          float64
rTeamColor          float64
bResult             float64
dtype: object

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28]))
model.add(tf.keras.layers.Dropout(rate=dropout_rate))
for _ in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_hidden_layer_neurons, activation=hidden_layer_activation_function))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

if op_type == OP_RMSPROP:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=optimizer_learning_rate, rho=gradient_decay_rho),
        metrics=["accuracy"])
else:
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.SGD(learning_rate=optimizer_learning_rate, decay=learning_rate_decay),
        metrics=["accuracy"])

In [12]:
y_train = train_data["bResult"]
y_test  =  test_data["bResult"]

In [13]:
y_train.head()

1    1.0
2    0.0
3    1.0
4    0.0
5    1.0
Name: bResult, dtype: float64

In [14]:
# Remove the match result column from the data.
X_train = train_data.drop("bResult", axis=1)
X_test  =  test_data.drop("bResult", axis=1)

In [15]:
X_train.head()

,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,rsPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,bCoopChampion,rCoopChampion,vsPlayer,vsChampion,bTeamColor,rTeamColor
1,0.289738,-0.257036,1.060552,0.377305,0.278699,-0.133841,0.350029,-0.112189,0.713630,-0.216989,...,1.126587,0.690359,0.441921,0.096551,0.134805,1.112304,-0.175703,0.510275,0.121245,-0.151772
2,-0.263885,-0.766333,-0.260659,-0.853495,-0.950215,-0.332546,3.260819,-1.208941,-0.316147,-1.474715,...,-0.030390,0.346412,-0.698948,-0.656359,-1.708774,0.131480,-1.581668,0.359287,-0.173614,2.230129
3,-0.358951,0.020762,0.699159,-0.409335,0.500586,0.974318,0.453523,0.311440,0.535215,-1.244525,...,-0.030390,0.870778,-0.171815,-0.720948,0.224274,-0.419115,0.677840,0.532583,1.890399,-1.884064
4,-0.712056,0.159661,-0.022286,-0.039075,0.478613,-0.214329,0.932187,-0.196032,3.315169,0.161031,...,-1.157802,0.870778,0.250172,1.716844,0.312525,0.042050,-0.818070,0.405424,-2.237627,2.230129
5,2.873311,0.535202,0.182033,1.240093,0.421766,-0.473280,-0.463145,-0.570401,-0.762098,-0.460782,...,-0.030390,0.346412,0.469980,-0.334529,0.588069,-0.805653,1.530957,1.675086,1.890399,-0.512666


In [16]:
import time
start_time = time.time()

In [17]:
# Train the model.

history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size)

Epoch 1/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.4599 - loss: 0.7964 - val_accuracy: 0.5607 - val_loss: 0.6784
Epoch 2/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5934 - loss: 0.6728 - val_accuracy: 0.6029 - val_loss: 0.6558
Epoch 3/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6549 - loss: 0.6145 - val_accuracy: 0.6240 - val_loss: 0.6455
Epoch 4/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6798 - loss: 0.5906 - val_accuracy: 0.6293 - val_loss: 0.6409
Epoch 5/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7140 - loss: 0.5541 - val_accuracy: 0.6372 - val_loss: 0.6392
Epoch 6/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7447 - loss: 0.5318 - val_accuracy: 0.6398 - val_loss: 0.6402
Epoch 7/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7487 - loss: 0.5141 - val_accuracy: 0.6425 - val_loss: 0.6425
Epoch 8/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7503 - loss: 0.5001 - val_accuracy: 0.6438 - val_loss: 0.6457


In [18]:
train_time = time.time() - start_time
train_time

6.73751163482666

In [19]:
import keras_tuner as kt

def build_model(hp):
    tune_n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=1)
    tune_n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    tune_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    tune_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.9)
    tune_optimizer = hp.Choice("optimizer", values=["sgd", "rmsprop"])
    
    tune_rho = hp.Float("rho", min_value=0.7, max_value=1.0)
    tune_learning_rate_decay = hp.Float("learning_rate_decay", min_value=0.01, max_value=0.99)
    tune_decay_steps = hp.Int("decay_steps", min_value=1000, max_value=100000)
    tune_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=tune_learning_rate,
        decay_steps=tune_decay_steps,
        decay_rate=tune_learning_rate_decay)
    
    if tune_optimizer == "sgd":
        tune_optimizer = tf.keras.optimizers.SGD(learning_rate=tune_lr_schedule)
    else:
        tune_optimizer = tf.keras.optimizers.RMSprop(learning_rate=tune_learning_rate, rho=tune_rho)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[28]))
    model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    for _ in range(tune_n_hidden):
        model.add(tf.keras.layers.Dense(tune_n_neurons, activation=hidden_layer_activation_function))
        model.add(tf.keras.layers.Dropout(rate=tune_dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation=output_layer_activation_function))

    model.compile(
        loss="binary_crossentropy",
        optimizer=tune_optimizer,
        metrics=["accuracy"])

    return model

In [21]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=10, overwrite=True, directory="results", project_name="tune_search", seed=42)
random_search_tuner.search(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=training_epochs,
    batch_size=batch_size
)

Trial 10 Complete [00h 00m 11s]
val_accuracy: 0.6503957509994507

Best val_accuracy So Far: 0.6715039610862732
Total elapsed time: 00h 01m 42s


In [ ]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [ ]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

In [ ]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

In [ ]:
best_trial.metrics.get_last_value("val_accuracy")

In [ ]:
#best_model.fit(X_train, y_train, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)